<a href="https://colab.research.google.com/github/MoroF10/enem2023-data-analysis/blob/main/03_An%C3%A1lise_Prescritiva_ENEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Prescritiva dos dados do ENEM

## **Objetivo do Projeto**<br>
Este notebook tem como foco a aplicação de modelos estatísticos para prever as notas de Matemática dos participantes do ENEM. A proposta é concentrar esforços exclusivamente na etapa de modelagem preditiva, utilizando dados que já foram tratados, limpos e explorados em notebooks anteriores disponíveis neste repositório.

Diferente de abordagens exploratórias, este projeto parte do pressuposto de que o entendimento dos dados — como distribuição, correlações e inconsistências — já foi realizado. O objetivo aqui é testar diferentes algoritmos de regressão, avaliar seus desempenhos e identificar o modelo mais eficiente para estimar as notas com base nas variáveis disponíveis.

A intenção é oferecer uma abordagem prática e objetiva, que possa ser útil em estudos educacionais, simulações ou como ferramenta de apoio à análise de desempenho.

## Estratégias Utilizadas na Análise <br>
Diante das características da base de dados — com muitas colunas, variáveis qualitativas e ausência de distribuição normal — algumas técnicas foram adotadas para otimizar o desempenho dos modelos:<br>
<br>
### Normalização<br>
Como as variáveis possuem escalas distintas (ex.: número de acertos, tempo de prova, renda familiar), será aplicada uma técnica de normalização. Isso garante que todas as variáveis estejam na mesma faixa de valores, evitando que atributos com maior magnitude dominem o processo de aprendizagem e favorecendo algoritmos sensíveis à escala.<br>
<br>
### PCA (Análise de Componentes Principais)<br>
Para lidar com a alta dimensionalidade, será utilizada a técnica de PCA com os seguintes objetivos:

. Eliminar redundâncias e correlações entre variáveis

. Reduzir o tempo de treinamento dos modelos

. Minimizar o risco de overfitting

. Preservar a maior parte da variância explicada pelos dados originais

Essa redução permite trabalhar com um conjunto de variáveis mais compacto e informativo, facilitando a construção de modelos mais eficientes e interpretáveis.<br>
<br>
### Transformação de Variáveis Qualitativas<br>

A base contém diversas colunas com atributos categóricos. Para convertê-las em variáveis numéricas, será utilizada a técnica de SimpleImputer do scikit-learn, evitando o uso de OneHotEncoder, que poderia gerar um conjunto de dados excessivamente grande e esparso. Essa escolha busca manter a eficiência computacional sem perder representatividade.

In [4]:
#Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format','{:.2f}'.format)

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [39]:
#Importando os dados
caminho = ("dados_amostra.csv")
df = pd.read_csv(caminho, sep=',', encoding='latin1')

In [3]:
df.head()


,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,n_faltas
0,210061362356,8,F,1,3,1,1,5,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3143302,Montes Claros,31,MG,1,1,1,1,481.30,295.50,503.80,383.00,1,1.00,120.00,120.00,120.00,120.00,80.00,560.00,H,H,A,A,3,B,A,B,D,B,A,B,A,A,A,A,A,A,A,A,B,D,A,A,B,4
1,210060716718,5,F,1,2,1,1,2,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3304557,Rio de Janeiro,33,RJ,1,1,1,1,590.60,604.80,543.30,650.40,0,1.00,120.00,40.00,40.00,80.00,40.00,320.00,E,D,C,B,3,C,A,C,C,A,A,B,B,B,A,B,A,A,B,A,A,D,A,B,B,4
2,210058482642,1,F,1,3,1,3,0,1,1.00,1,0.00,0,0.00,0,2.00,1.00,0.00,2918209,JiquiriÃÂ§ÃÂ¡,29,BA,1,1,1,1,421.50,489.40,444.10,397.00,0,1.00,80.00,100.00,80.00,100.00,80.00,440.00,B,E,A,A,3,B,A,B,B,A,A,B,A,A,A,A,A,A,A,A,A,B,A,A,A,4
3,210059249770,3,F,1,1,1,1,1,1,1.00,0,0.00,0,0.00,0,2.00,1.00,0.00,3534401,Osasco,35,SP,1,1,1,1,594.30,618.20,662.10,670.10,0,1.00,160.00,200.00,180.00,180.00,180.00,900.00,D,G,D,E,2,O,A,C,D,B,A,B,B,B,B,B,B,B,C,A,B,C,A,C,B,4
4,210059793602,2,F,0,0,1,2,0,2,1.00,0,2307650.00,MaracanaÃÂº,23.00,CE,2.00,1.00,1.00,2307650,MaracanaÃÂº,23,CE,1,1,1,1,0.00,360.00,565.10,0.00,1,1.00,120.00,120.00,120.00,140.00,80.00,580.00,B,E,C,B,5,D,A,C,D,B,A,B,A,B,A,B,A,A,B,B,A,D,A,A,B,4


A etapa inicial do processamento dos dados será a conversão de variáveis qualitativas em valores numéricos. Essa transformação é essencial para garantir o funcionamento adequado dos modelos de machine learning. Para isso, utilizarei a biblioteca scikit-learn, especificamente o OrdinalEncoder.

In [42]:
#Criando uma lista com todos os valores categoricos
cat_cols = df.select_dtypes(include=['object', 'category']).columns

#criando uma instância para Label Enconder
le = LabelEncoder()

#Aplicando Label Enconder em todos os dados categoricos
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])